In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import re

In [3]:
train_df = pd.read_csv("Divar.csv")

C:\Users\USER\AppData\Local\Temp\ipykernel_6620\960344712.py:1: DtypeWarning: Columns (11,27,29,53) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("Divar.csv")


In [16]:
df_subset = pd.concat([train_df.iloc[:, :9], train_df.iloc[:, 58:]], axis=1)
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Unnamed: 0          1000000 non-null  int64  
 1   cat2_slug           1000000 non-null  object 
 2   cat3_slug           999999 non-null   object 
 3   city_slug           999998 non-null   object 
 4   neighborhood_slug   437139 non-null   object 
 5   created_at_month    1000000 non-null  object 
 6   user_type           288882 non-null   object 
 7   description         1000000 non-null  object 
 8   title               999946 non-null   object 
 9   location_latitude   655608 non-null   float64
 10  location_longitude  655608 non-null   float64
 11  location_radius     339699 non-null   float64
dtypes: float64(3), int64(1), object(8)
memory usage: 91.6+ MB


### Location

In [17]:
import utm

latitudes = df_subset["location_latitude"].values
longitudes = df_subset["location_longitude"].values

utm_coords = [utm.from_latlon(lat, lon) if not np.isnan(lat) and not np.isnan(lon) else (np.nan, np.nan, np.nan, np.nan)
              for lat, lon in zip(latitudes, longitudes)]

df_subset["utm_easting"] = [coord[0] for coord in utm_coords]
df_subset["utm_northing"] = [coord[1] for coord in utm_coords]

In [18]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Unnamed: 0          1000000 non-null  int64  
 1   cat2_slug           1000000 non-null  object 
 2   cat3_slug           999999 non-null   object 
 3   city_slug           999998 non-null   object 
 4   neighborhood_slug   437139 non-null   object 
 5   created_at_month    1000000 non-null  object 
 6   user_type           288882 non-null   object 
 7   description         1000000 non-null  object 
 8   title               999946 non-null   object 
 9   location_latitude   655608 non-null   float64
 10  location_longitude  655608 non-null   float64
 11  location_radius     339699 non-null   float64
 12  utm_easting         655608 non-null   float64
 13  utm_northing        655608 non-null   float64
dtypes: float64(5), int64(1), object(8)
memory usage: 106.8+ MB


In [19]:
from sklearn.neighbors import NearestNeighbors

valid_mask = df_subset["neighborhood_slug"].notna()
coords_valid = df_subset.loc[valid_mask, ["utm_easting", "utm_northing"]].dropna()
labels_valid = df_subset.loc[valid_mask, "neighborhood_slug"]

knn = NearestNeighbors(n_neighbors=5)
knn.fit(coords_valid)

missing_mask = df_subset["neighborhood_slug"].isna()
coords_missing = df_subset.loc[missing_mask, ["utm_easting", "utm_northing"]].dropna()

distances, indices = knn.kneighbors(coords_missing)

filled_labels = []
for neighbor_idxs in indices:
    neighbor_labels = labels_valid.iloc[neighbor_idxs]
    most_common = neighbor_labels.mode()[0]
    filled_labels.append(most_common)

df_subset.loc[coords_missing.index, "neighborhood_slug"] = filled_labels

df_subset.neighborhood_slug.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: neighborhood_slug
Non-Null Count   Dtype 
--------------   ----- 
800249 non-null  object
dtypes: object(1)
memory usage: 7.6+ MB


In [20]:
nan_neighborhood_cities = (df_subset.groupby("city_slug")["neighborhood_slug"].apply(lambda x: x.isna().all()))

# Filter only those cities
nan_neighborhood_cities = nan_neighborhood_cities[nan_neighborhood_cities]

print(f"Number of such cities: {len(nan_neighborhood_cities)}")
print(nan_neighborhood_cities.index.tolist())

Number of such cities: 0
[]


In [21]:
city_mode = (df_subset.groupby("city_slug")["neighborhood_slug"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan))

# Fill NaNs using city mode
df_subset["neighborhood_slug"] = df_subset.apply(
    lambda row: city_mode[row["city_slug"]]
    if pd.isna(row["neighborhood_slug"]) and row["city_slug"] in city_mode
    else row["neighborhood_slug"], axis=1)

df_subset["neighborhood_slug"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1000000 entries, 0 to 999999
Series name: neighborhood_slug
Non-Null Count   Dtype 
--------------   ----- 
999998 non-null  object
dtypes: object(1)
memory usage: 7.6+ MB


In [ ]:
x = df_subset.loc[df_subset["Unnamed: 0"] == 22	, "description"].iloc[0]
y = df_subset.loc[df_subset["Unnamed: 0"] == 97	, "description"].iloc[0]

print(x)
# print(y)

زمین واقع در کلاته گیزی 
پنج کیلو متر بعد از ابراهیم اباد جاده اسفالت 
از لب جاده ۱۰۰۰ متر فاصله زمین 
زمین کاملا مسطح و صاف شده 
راه کشی شده قطعه مشخص 
اماده قطعه بندی 
چهارمیخ 
تمامی نامه ها تاییدیه ها سورا و دهیاری تقسیم نامه ها قولنامه ها موجود و مرتب 
زمین مزروعی 
معاوضه با اپارتمان الهیه امامیه و…


In [22]:
cols_to_concat = ["Unnamed: 0", "cat3_slug", "city_slug", "neighborhood_slug"]
df_location = df_subset[cols_to_concat].copy()
df_location.to_csv("E:\\AI_Quera\\Real-estate_PricePrediction\\processed data\\location.csv", index=False)
df_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   Unnamed: 0         1000000 non-null  int64 
 1   cat3_slug          999999 non-null   object
 2   city_slug          999998 non-null   object
 3   neighborhood_slug  999998 non-null   object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
